In [153]:
import re
import time
import json
import redis
import scrapy
import datetime
import traceback
import pandas as pd
import codecs
from lxml import etree

from sqlalchemy import create_engine
from selenium import webdriver

# import the necessary packages
from imutils.object_detection import non_max_suppression
import numpy as np
import pytesseract
import argparse
import cv2

pool = redis.ConnectionPool(host='172.17.0.2', port=6379, decode_responses=True, db=1, password='De32wsxC')
redis_con = redis.Redis(connection_pool=pool)

PRODUCE_DB_ADDR_OUTTER = '172.17.0.4'
PRODUCE_DB_USER = 'root'
PRODUCE_DB_PASSWD = '123456'

LCCAL_ENGINE = 'mysql+pymysql://'+PRODUCE_DB_USER+':'+PRODUCE_DB_PASSWD+'@'+PRODUCE_DB_ADDR_OUTTER+'/temp?charset=utf8'
engine = create_engine(LCCAL_ENGINE)

In [125]:
def price_recognition(image,net):
    orig = image.copy()
    (origH, origW) = image.shape[:2]

    # extract the actual padded ROI
    roi = orig[0:origH, 0:origW]
    config = ("-l eng --oem 1 --psm 7")
    text = pytesseract.image_to_string(roi, config=config)
    
    return text

def assemble_series_list(brand):
    """
    组合series list
    """
    # 清空数据库
    redis_con.flushdb()
    for i in range(0, len(brand)):
        url = 'https://ssl-meta.che300.com/meta/series/series_brand' + str(brand['brand_id'][i]) + '.json?v=1561446955'
        redis_con.sadd('che300_series_list', url)
        
def assemble_detail_list(model):
    """
    组合detail list
    """
    # 清空数据库
    redis_con.flushdb()
    for i in range(0, len(model)):
        url = 'https://ssl-meta.che300.com/meta/model/model_series' + str(model['series_id'][i]) + '.json?v=1561446955'
        redis_con.sadd('che300_detail_list', url)

In [86]:
driver = webdriver.Firefox()

urls = {
    'home': 'https://www.che300.com/pinggu?from=bd_seo&rt=1561513934702'
}
driver.get(urls['home'])
driver.find_elements_by_xpath("//div[@class='select_box']")[0].click()

## 一.品牌抓取

In [87]:
brand = pd.DataFrame([],columns=['brand_id','brand_name','first_letter'])
elem=driver.find_elements_by_xpath("//p[@class='pinpailist list_1']")

for i in range(0,len(elem)):
    brand.loc[i,'brand_id'] = elem[i].get_attribute('id')
    brand['brand_name'][i] = elem[i].text
    brand['first_letter'][i] = elem[i].get_attribute('rel')
brand.to_csv('./brand_che300.csv', index=False)
assemble_series_list(brand)
model = pd.DataFrame([],columns=['brand_id','series_id','series_name','is_green','series_group_name'])

## 二.车型抓取

In [88]:
while True:
    try:
        url = redis_con.spop('che300_series_list')
        if url == None:
            break
        driver.get(url)
        time.sleep(3)
        count_series = len(driver.find_elements_by_xpath("//span[@class='treeLabel objectLabel']"))
        if count_series == 0:
            continue

        result = driver.find_elements_by_xpath("//span[@class='objectBox objectBox-string']")
        temp = pd.DataFrame([],columns=['brand_id','series_id','series_name','is_green','series_group_name'])
        for i in range(0,count_series):
            temp.loc[i,'brand_id'] = int(re.findall('brand(.*)\.json',url)[0])
            temp.loc[i,'series_id'] = result[i*4].text.replace("\"","")
            temp.loc[i,'series_name'] = result[i*4+1].text.replace("\"","")
            temp.loc[i,'is_green'] = result[i*4+2].text.replace("\"","")
            temp.loc[i,'series_group_name'] = result[i*4+3].text.replace("\"","")
        model = model.append(temp,sort=False).reset_index(drop=True)
    except Exception:
        redis_con.sadd('che300_series_list', url)
        raise Exception(traceback.format_exc())
model.to_csv('./model_che300.csv', index=False)
assemble_detail_list(model)
detail = pd.DataFrame([],columns=['series_id','model_id','model_name','model_price','model_year','min_reg_year',
                                  'max_reg_year','liter','liter_type','gear_type','discharge_standard','is_green'])

## 三.款型抓取

In [137]:
assemble_detail_list(model)
detail = pd.DataFrame([],columns=['series_id','model_id','model_name','model_price','model_year','min_reg_year',
                                  'max_reg_year','liter','liter_type','gear_type','discharge_standard','is_green'])

In [140]:
firefox_profile = webdriver.FirefoxProfile()
firefox_profile.set_preference('permissions.default.stylesheet', 2)
firefox_profile.set_preference('permissions.default.image', 2)
firefox_profile.set_preference('devtools.jsonview.enabled', 'false')

driver = webdriver.Firefox(firefox_profile=firefox_profile)

In [141]:
while True:
    try:
        url = redis_con.spop('che300_detail_list')
        if url == None:
            break
        driver.get(url)
        time.sleep(3)
        result = driver.find_elements_by_xpath("//pre")[0].text
        result = eval(result)
        count_details = len(result)
        if count_details == 0:
            continue

        temp = pd.DataFrame([],columns=['series_id','model_id','model_name','model_price','model_year','min_reg_year',
                                  'max_reg_year','liter','liter_type','gear_type','discharge_standard','is_green'])
        for i in range(0,count_details):
            temp.loc[i,'series_id'] = int(re.findall('series(.*)\.json',url)[0])
            temp.loc[i,'model_id'] = result[i]['model_id']
            temp.loc[i,'model_name'] = result[i]['model_name']
            temp.loc[i,'model_price'] = result[i]['model_price']
            temp.loc[i,'model_year'] = result[i]['model_year']
            temp.loc[i,'min_reg_year'] = result[i]['min_reg_year']
            temp.loc[i,'max_reg_year'] = result[i]['max_reg_year']
            temp.loc[i,'liter'] = result[i]['liter']
            temp.loc[i,'liter_type'] = result[i]['liter_type']
            temp.loc[i,'gear_type'] = result[i]['gear_type']
            temp.loc[i,'discharge_standard'] = result[i]['discharge_standard']
            temp.loc[i,'is_green'] = result[i]['is_green']
        detail = detail.append(temp,sort=False).reset_index(drop=True)
    except Exception:
        redis_con.sadd('che300_detail_list', url)
        raise Exception(traceback.format_exc())
detail.to_csv('./detail_che300.csv', index=False)

In [153]:
brand = pd.read_csv('./brand_che300.csv')
model = pd.read_csv('./model_che300.csv')
detail = pd.read_csv('./detail_che300.csv')

model = model.merge(brand,how='left',on=['brand_id'])
detail = detail.merge(model.drop(['is_green'],axis=1),how='left',on=['series_id'])
detail.to_csv('/home/ml/ProgramProjects/detail_match/jupyter/detail_che300.csv',index=False)

## 四.估值抓取

In [150]:
def assemble_valute_detail_list(df):
    url = 'https://www.che300.com/pinggu/v11c125m'+str(int(df['model_id']))+'r'+str(df['online_year'])+'-3'+'g'+str(df['mile'])+'?click=homepage&rt=1561682411644'
    redis_con.sadd('che300_valute_detail_list', url)
    return url

che300_match_result = pd.read_csv('./che300_match_result.csv')
che300_match_result = che300_match_result.loc[:,['model_id','origin_name','detail_slug','cos_similar']]

car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')
car_autohome_all['used_years'] = datetime.datetime.now().year - car_autohome_all['online_year']
car_autohome_all.loc[(car_autohome_all['used_years'] < 0), 'used_years'] = 0
car_autohome_all = car_autohome_all.sort_values(by=['brand_slug', 'model_slug', 'online_year', 'price_bn']).reset_index(drop=True)

at = car_autohome_all.loc[(car_autohome_all['control'] == '自动'),:].reset_index(drop=True)
mt = car_autohome_all.loc[(car_autohome_all['control'] == '手动'),:].reset_index(drop=True)

# 取低配数据
at_low_config_car = at.loc[at.groupby(['brand_slug', 'model_slug', 'online_year']).price_bn.idxmin(), :].reset_index(drop=True)
mt_low_config_car = mt.loc[mt.groupby(['brand_slug', 'model_slug', 'online_year']).price_bn.idxmin(), :].reset_index(drop=True)
low_config_car = at_low_config_car.append(mt_low_config_car,sort=False)
print(len(low_config_car))
low_config_car = low_config_car.merge(che300_match_result,how='left',on=['detail_slug'])

miss_match = low_config_car.loc[(low_config_car['model_id'].isnull()),:].reset_index(drop=True)
miss_match.to_csv('./che300_miss_match.csv',index=False)
print(len(miss_match))

low_config_car = low_config_car.loc[(low_config_car['model_id'].notnull()),:].reset_index(drop=True)
low_config_car = low_config_car.loc[low_config_car.groupby(['detail_slug']).cos_similar.idxmax(), :].reset_index(drop=True)
print(len(low_config_car))

low_config_car['mile'] = low_config_car['used_years'] * 2
low_config_car.loc[(low_config_car['used_years'] == 0),'mile'] = 0.5
low_config_car['url'] = low_config_car.apply(assemble_valute_detail_list,axis=1)

7904
582
7322


In [151]:
driver = webdriver.Firefox()
driver.get('https://www.che300.com/pinggu?from=bd_seo&rt=1561680317573')

In [165]:
# load the pre-trained EAST text detector
net = cv2.dnn.readNet('opencv-text-recognition/frozen_east_text_detection.pb')

while True:
    try:
        url = redis_con.spop('che300_valute_detail_list')
        if url == None:
            break
        driver.get(url)
        time.sleep(1)
        
        sp_up = driver.find_elements_by_xpath("//ul[@class='sp-value clearfix']/li/span/img")
        sp_bottom = driver.find_elements_by_xpath("//div[@class='sp-price']/ul[@class='sp-bottom']/li/img")
        
        if len(sp_up) != 9:
            raise Exception('异常')
        
        up_price_recognition = []
        bottom_price_recognition = []
        for i,img in enumerate(sp_up):
            image = img.get_attribute('src').split('base64,')[1]
            image = base64.b64decode(image)
            image_result = open('images/up_'+str(i)+'.png', 'wb')
            image_result.write(image)
            image_result.flush()
            image_result.close()
            # load the input image and grab the image dimensions
            image = cv2.imread('images/up_'+str(i)+'.png')
            price = price_recognition(image,net)
            up_price_recognition.append(float(price))

        for i,img in enumerate(sp_bottom):
            image = img.get_attribute('src').split('base64,')[1]
            image = base64.b64decode(image)
            image_result = open('images/bottom_'+str(i)+'.png', 'wb')
            image_result.write(image)
            image_result.flush()
            image_result.close()
            # load the input image and grab the image dimensions
            image = cv2.imread('images/bottom_'+str(i)+'.png')
            price = price_recognition(image,net)
            bottom_price_recognition.append(float(price))
    
        temp = pd.DataFrame([],columns=['model_id','up_price','bottom_price'])
        temp.loc[i,'model_id'] = str(int(re.findall('v11c125m(.*)r20',url)[0]))
        temp.loc[i,'up_price'] = str(up_price_recognition)
        temp.loc[i,'bottom_price'] = str(bottom_price_recognition)
        temp.to_sql(name='che300_valuate', if_exists='append', con=engine, index=False)
    except Exception:
        redis_con.sadd('che300_valute_detail_list', url)
        raise Exception(traceback.format_exc())
valuate.to_csv('./che300_valuate.csv', index=False)

Exception: Traceback (most recent call last):
  File "<ipython-input-165-f66222bac725>", line 16, in <module>
    raise Exception('异常')
Exception: 异常
